In [1]:
import pandas as pd
import numpy as np

In [2]:
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
symbol='TATACOFFEE.NS'
start = dt.datetime(2013,6,1)
end = dt.datetime(2022,2,11)
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2144, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1715
X_train length: (1715, 4)
X_test length: (429, 4)
y_train length: (1715, 4)
y_test length: (429, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [9]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

In [10]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def holt_winters_exponential_smoothing(dataset, column_name, seasonal, seasonal_periods):
    dataset_hwes = dataset[column_name]
    test_obs_hwes = 28
    train_hwes = dataset_hwes[:-test_obs_hwes]
    test_hwes = dataset_hwes[-test_obs_hwes:]

    model_hwes = ExponentialSmoothing(train_hwes, seasonal=seasonal, seasonal_periods=seasonal_periods)
    results_hwes = model_hwes.fit()

    # Forecasting
    pred_hwes = results_hwes.forecast(steps=28)
    pred_hwes.to_csv("hwes_forecasted_{}.csv".format(test_obs_hwes))

    # Evaluating performance
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    rmse_hwes = round(mean_squared_error(test_hwes, pred_hwes, squared=False), 2)
    mape_hwes = mean_absolute_percentage_error(test_hwes, pred_hwes)

    performance["Model"].append("HWES")
    performance["RMSE"].append(rmse_hwes)
    performance["MaPe"].append(mape_hwes)
    performance["Lag"].append(None)
    performance["Test"].append(test_obs_hwes)

    perf_hwes = pd.DataFrame(performance)
    return perf_hwes, results_hwes, pred_hwes


In [11]:
column_hwes = "Close"
seasonal_hwes = "add"  # Choosing "add" or "multiplicative" based on our data

In [12]:
seasonal_periods_hwes = 7  # Setting this based on our data's seasonality
perf_hwes, result_hwes, pred_hwes = holt_winters_exponential_smoothing(data1, column_hwes, seasonal_hwes, seasonal_periods_hwes)

In [13]:
data1

,Open,High,Low,Close
0,0.517223,0.485749,0.529877,0.522210
1,0.500522,0.491400,0.530969,0.519394
2,0.498434,0.472236,0.528786,0.517972
3,0.506785,0.472236,0.532251,0.520225
4,0.507411,0.479533,0.527149,0.513788
...,...,...,...,...
2139,0.877349,0.819410,0.900409,0.880633
2140,0.856733,0.807862,0.885130,0.870440
2141,0.848643,0.799017,0.873670,0.865612
2142,0.864301,0.810565,0.892224,0.873927


In [14]:
perf_hwes

,Model,RMSE,MaPe,Lag,Test
0,HWES,0.02,1.976625,None,28


In [ ]:
#pip install --upgrade scikit-learn

# Problems Encountered

Codes created problems -     

mape_hwes = mean_absolute_percentage_error(test_hwes, pred_hwes) &      perf_hwes, result_hwes, pred_hwes = holt_winters_exponential_smoothing(data1, column_hwes, seasonal_hwes, seasonal_periods_hwes)

Reason : The mean_absolute_percentage_error function is not a part of scikit-learn in versions prior to 0.24.0, and its absence is causing the error. So implemented function for mean_absolute_percentage_error as below.

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
